## NNCL ASSIGNMENT 3

In [8]:
import numpy as np
import pandas as pd

In [41]:
class Network:
    def __init__(self, P, K, learning_rate):
        # P = amount of input samples
        self.P = P
        # K = amount of neurons in the hidden layer
        self.K = K
        # eta = learning rate
        self.eta = learning_rate
        # W = weights form input to hidden layers
        self.W = self.initWeights1()
        # V = weights from hidden layer to output, fixed to 1
        self.V = np.ones((1, self.K))

    def initWeights1(self):
        
        # Generate random vectors and normalize each vector to have a norm of 1
        weights = np.random.randn(self.P, self.K)
        norms_squared = np.linalg.norm(weights, axis=1, keepdims=True)**2
        normalized_weights = weights / norms_squared

        return normalized_weights
    
    def forwardPass(self, x):
        """
        Tanh activation function. 
        """
        
        # Calculate the dot product of the first-layer weights and the input
        dot_product = np.dot(self.W, x)
        # Apply hyperbolic tangent element-wise and sum for sigma. 
        tanh_result = np.tanh(dot_product)
        sigma = np.sum(self.V * tanh_result)

        # Sigma is the output of the network for a given input x
        return sigma

    def stochasticGradientDescent(self, sigma, xi, tau):
        """
        Stochastic gradient descent
        """
        # Use the gradient with respect to its contribution to the error
        gradient = (sigma - tau) * (1 - np.tanh(np.dot(self.W, xi))**2)
        # Update the weights
        self.W = self.W - self.eta * gradient * xi
    
    def calculateError(self, sigma, tau):
        # Error is the quadratic difference between sigma (network output) and
        # tau (target value)
        return ((sigma - tau)**2)/2


    def train(self, t_max, dataset, test_set):
        """
        Train the network using stochastic gradient descent. 
        """
        # Select a random sample from the dataset, and perform a forward pass.
        # Then, update the weights using the SGD algorithm.
        # Run for t_max * P iterations.
        # Select a random sample from the dataset, make sure that for each t,
        # all samples are used, but in random order.
        for epoch in range(t_max):
            # For each epoch, keep track of the error and print the average
            # error for the epoch.
            epoch_error = 0
            epoch_error_test = 0
            for p in np.random.permutation(len(dataset)):
                xi, tau = dataset[p]
                sigma = self.forwardPass(xi)
                epoch_error += self.calculateError(sigma, tau)
                self.stochasticGradientDescent(sigma, xi, tau)
            epoch_error /= len(dataset)

            for p in np.random.permutation(len(test_set)):
                xi, tau = test_set[p]
                sigma = self.forwardPass(xi)
                epoch_error_test += self.calculateError(sigma, tau)
            epoch_error_test /= len(test_set)

            print("Epoch: {}, Error: {} Test Error: {}".format(epoch, epoch_error, epoch_error_test))


# Inputs
xi = pd.read_csv("data/xi.csv", delimiter=',', header=None)
# Labels
tau = pd.read_csv("data/tau.csv", delimiter=',', header=None)
dataset = list(zip(xi, tau))

# n = amount of input samples
P = len(xi)
K = 2

# Take only the first 100 samples
trainset = dataset[:4000]
testset = dataset[4000:]
print(len(trainset))
print(len(testset))

network = Network(P=P, K=K, learning_rate=0.01)
network.train(t_max=100, dataset=trainset, test_set=testset)

4000
1000
Epoch: 0, Error: 2640579.6075764215 Test Error: 10110494.75
Epoch: 1, Error: 2641794.0824345886 Test Error: 10110494.75
Epoch: 2, Error: 2641744.6796607967 Test Error: 10110494.75
Epoch: 3, Error: 2641744.676480645 Test Error: 10110494.75
Epoch: 4, Error: 2641744.6727463747 Test Error: 10110494.75
Epoch: 5, Error: 2641744.673330866 Test Error: 10110494.75
Epoch: 6, Error: 2641744.6742991596 Test Error: 10110494.75
Epoch: 7, Error: 2641744.672687424 Test Error: 10110494.75
Epoch: 8, Error: 2641744.6750150733 Test Error: 10110494.75
Epoch: 9, Error: 2641744.6733527486 Test Error: 10110494.75
Epoch: 10, Error: 2641744.673716022 Test Error: 10110494.75
Epoch: 11, Error: 2641744.6733257207 Test Error: 10110494.75
Epoch: 12, Error: 2641744.673712588 Test Error: 10110494.75
Epoch: 13, Error: 2641744.673824577 Test Error: 10110494.75
Epoch: 14, Error: 2641744.673449732 Test Error: 10110494.75
Epoch: 15, Error: 2641744.6736809146 Test Error: 10110494.75
Epoch: 16, Error: 2641744.67364